In [53]:
# Import modules
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gmaps
import pyproj
import googlemaps

In [54]:
gmaps_api_key =  ''
gmaps = googlemaps.Client(key=gmaps_api_key)

In [131]:
# Load Sampled Data
df=pd.read_csv("Simplified_Citation_After_2015July_to_2019.csv",low_memory=False)
df.head(5)

,Unnamed: 0,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
0,28,4361326712,2019/09/17 12:00:00 AM,1205.0,NaN,NaN,CA,NaN,TOYT,PA,GY,309 WINDWARD AVE,00163,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.419487e+06,1.818861e+06
1,112,4361286823,2019/09/17 12:00:00 AM,853.0,NaN,NaN,CA,NaN,TOYT,PA,WT,934 84TH ST W,00553,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.473690e+06,1.808839e+06
2,224,4361159515,2019/09/17 12:00:00 AM,833.0,NaN,NaN,CA,NaN,NISS,PA,WT,2601 MONMOUTH AVE,00536,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.475329e+06,1.834194e+06
3,473,4361237226,2019/09/17 12:00:00 AM,1015.0,NaN,NaN,CA,NaN,HOND,PA,RD,615 HAMPTON DR,00133,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.417752e+06,1.820642e+06
4,592,4360129196,2019/09/17 12:00:00 AM,1007.0,NaN,NaN,CA,NaN,VOLV,PA,SL,1346 MCCADDEN PL N,00487,54.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.459495e+06,1.857205e+06


In [132]:
df.columns

Index(['Unnamed: 0', 'Ticket number', 'Issue Date', 'Issue time', 'Meter Id',
       'Marked Time', 'RP State Plate', 'VIN', 'Make', 'Body Style', 'Color',
       'Location', 'Route', 'Agency', 'Violation code',
       'Violation Description', 'Fine amount', 'Latitude', 'Longitude'],
      dtype='object')

In [133]:
# Identify bad Longitude/Latitude values
df['Longitude'].value_counts()

9.999900e+04    11416
1.819688e+06       98
1.859071e+06       87
1.882602e+06       64
1.849114e+06       49
                ...  
1.843271e+06        1
1.845315e+06        1
1.909664e+06        1
1.835485e+06        1
1.857892e+06        1
Name: Longitude, Length: 50929, dtype: int64

In [134]:
# Check for null values
df.isnull().sum()

Unnamed: 0                   0
Ticket number                0
Issue Date                   0
Issue time                  15
Meter Id                 65438
Marked Time              84748
RP State Plate               2
VIN                      87554
Make                        71
Body Style                 115
Color                       25
Location                     2
Route                      636
Agency                       0
Violation code               0
Violation Description        5
Fine amount                 52
Latitude                     0
Longitude                    0
dtype: int64

In [135]:
# Remove null Location records
df = df[~df.Location.isnull()]

In [136]:
# Fraction of records with bad Lat/Lon values
len(df[(df['Latitude'] == 99999) | (df['Longitude'] == 99999)])/len(df)

0.13036823830409358

In [137]:
# New dataset statistics
df[['Latitude', 'Longitude']].describe()

,Latitude,Longitude
count,8.755200e+04,8.755200e+04
mean,5.627169e+06,1.620580e+06
std,2.140200e+06,5.893556e+05
min,9.999900e+04,9.999900e+04
25%,6.423939e+06,1.826010e+06
50%,6.453559e+06,1.842360e+06
75%,6.475588e+06,1.858205e+06
max,6.513150e+06,1.941866e+06


In [144]:
# Mask to work on good data first
mask = (df['Latitude'] != 99999) | (df['Longitude'] != 99999)
df[mask].head()

,Unnamed: 0,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
0,28,4361326712,2019/09/17 12:00:00 AM,1205.0,NaN,NaN,CA,NaN,TOYT,PA,GY,309 WINDWARD AVE,00163,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.419487e+06,1.818861e+06
1,112,4361286823,2019/09/17 12:00:00 AM,853.0,NaN,NaN,CA,NaN,TOYT,PA,WT,934 84TH ST W,00553,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.473690e+06,1.808839e+06
2,224,4361159515,2019/09/17 12:00:00 AM,833.0,NaN,NaN,CA,NaN,NISS,PA,WT,2601 MONMOUTH AVE,00536,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.475329e+06,1.834194e+06
3,473,4361237226,2019/09/17 12:00:00 AM,1015.0,NaN,NaN,CA,NaN,HOND,PA,RD,615 HAMPTON DR,00133,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.417752e+06,1.820642e+06
4,592,4360129196,2019/09/17 12:00:00 AM,1007.0,NaN,NaN,CA,NaN,VOLV,PA,SL,1346 MCCADDEN PL N,00487,54.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.459495e+06,1.857205e+06


In [151]:
df.loc[mask,['Latitude','Longitude']].to_numpy()

array([[  33.9894116 , -118.4690242 ],
       [  33.96243547, -118.29012558],
       [  34.03212   , -118.284957  ],
       ...,
       [  33.96075103, -118.43936151],
       [  33.95891498, -118.44837753],
       [  34.03966412, -118.32245006]])

In [154]:
# Conversion using pyproj module
inProj = pyproj.Proj('epsg:2229', preserve_units=True) # make sure unit preserve 
outProj = pyproj.Proj('epsg:4326')
df.loc[mask,'Latitude'], df.loc[mask,'Longitude'] = pyproj.transform(inProj,outProj, df[mask]['Latitude'].values, df[mask]['Longitude'].values)

In [158]:
df[mask].head()

,Unnamed: 0,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
0,28,4361326712,2019/09/17 12:00:00 AM,1205.0,NaN,NaN,CA,NaN,TOYT,PA,GY,309 WINDWARD AVE,00163,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,27.195712,-138.152012
1,112,4361286823,2019/09/17 12:00:00 AM,853.0,NaN,NaN,CA,NaN,TOYT,PA,WT,934 84TH ST W,00553,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,27.195712,-138.152012
2,224,4361159515,2019/09/17 12:00:00 AM,833.0,NaN,NaN,CA,NaN,NISS,PA,WT,2601 MONMOUTH AVE,00536,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,27.195712,-138.152012
3,473,4361237226,2019/09/17 12:00:00 AM,1015.0,NaN,NaN,CA,NaN,HOND,PA,RD,615 HAMPTON DR,00133,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,27.195712,-138.152012
4,592,4360129196,2019/09/17 12:00:00 AM,1007.0,NaN,NaN,CA,NaN,VOLV,PA,SL,1346 MCCADDEN PL N,00487,54.0,80.69BS,NO PARK/STREET CLEAN,73.0,27.195712,-138.152012


In [159]:
def geocode_9s(location):
    result = gmaps.geocode('{}, Los Angeles, CA'.format(location))
    return result[0].get('geometry').get('location').get('lat'), result[0].get('geometry').get('location').get('lng')

In [172]:
df

,Unnamed: 0,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
0,28,4361326712,2019/09/17 12:00:00 AM,1205.0,NaN,NaN,CA,NaN,TOYT,PA,GY,309 WINDWARD AVE,00163,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,27.195712,-138.152012
1,112,4361286823,2019/09/17 12:00:00 AM,853.0,NaN,NaN,CA,NaN,TOYT,PA,WT,934 84TH ST W,00553,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,27.195712,-138.152012
2,224,4361159515,2019/09/17 12:00:00 AM,833.0,NaN,NaN,CA,NaN,NISS,PA,WT,2601 MONMOUTH AVE,00536,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,27.195712,-138.152012
3,473,4361237226,2019/09/17 12:00:00 AM,1015.0,NaN,NaN,CA,NaN,HOND,PA,RD,615 HAMPTON DR,00133,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,27.195712,-138.152012
4,592,4360129196,2019/09/17 12:00:00 AM,1007.0,NaN,NaN,CA,NaN,VOLV,PA,SL,1346 MCCADDEN PL N,00487,54.0,80.69BS,NO PARK/STREET CLEAN,73.0,27.195712,-138.152012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87549,8705615,4262938105,2015/07/02 12:00:00 AM,1622.0,BH198,NaN,CA,NaN,NISS,PA,SL,200 BREED ST N,00602,56.0,88.13B+,METER EXP.,63.0,27.195712,-138.152012
87550,8705782,4262766314,2015/07/02 12:00:00 AM,1302.0,NaN,NaN,CA,NaN,CHEV,PA,GN,5306 VANTAGE AVE,00355,53.0,80.69BS,NO PARK/STREET CLEAN,73.0,27.195712,-138.152012
87551,8705813,4262762140,2015/07/02 12:00:00 AM,1325.0,NaN,NaN,CA,NaN,NISS,PA,BK,8229 CALABAR AVE,00137,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,27.195712,-138.152012
87552,8705825,4262762022,2015/07/02 12:00:00 AM,1256.0,NaN,NaN,CA,NaN,NISS,PA,BK,R/O 203 CULVER BLVD E,00137,51.0,80.69AP+,NO STOP/STANDING,93.0,27.195712,-138.152012


In [160]:
converted_lat, converted_lon = df.loc[~mask, 'Location'].apply(geocode_9s)

KeyboardInterrupt: 

In [162]:
converted_lat

NameError: name 'converted_lat' is not defined

In [10]:
#picked 100000 samples since my computer cannot process more data than that 
gmaps.configure(api_key='') # type in your own API 
morning_time = (df["Issue time"] > 600) & (df["Issue time"] <= 1200) & (df.iloc[:100000]["Violation Description"] == "RED ZONE")
morning_time_df = df[morning_time][["Latitude_new","Longitude_new"]]

afternoon_time = (df["Issue time"] > 1200) & (df["Issue time"] <= 1800) & (df.iloc[:100000]["Violation Description"] == "RED ZONE")
afternoon_time_df = df[afternoon_time][["Latitude_new","Longitude_new"]]

evening_time = (df["Issue time"] > 1800)  & (df.iloc[:100000]["Violation Description"] == "RED ZONE")
evening_time_df = df[evening_time][["Latitude_new","Longitude_new"]]

fig = gmaps.figure()
morning_layer = gmaps.symbol_layer(morning_time_df, fill_color = "red", stroke_color = "red", scale = 1)
afternoon_layer = gmaps.symbol_layer(afternoon_time_df, fill_color = "green", stroke_color = "green", scale = 1)
evening_layer = gmaps.symbol_layer(evening_time_df, fill_color = "blue", stroke_color = "blue", scale = 1)
fig.add_layer(morning_layer)
fig.add_layer(afternoon_layer)
fig.add_layer(evening_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [128]:
df['Violation code'][df['Violation Description'] == 'NO PARK/STREET CLEAN'].value_counts()

80.69BS    21208
8069BS       491
Name: Violation code, dtype: int64

In [199]:
df['Violation Description'].unique()

array(['NO PARK/STREET CLEAN', 'EXCEED 72HRS-ST', 'RED ZONE',
       'NO STOP/STAND', 'STANDNG IN ALLEY', 'DISPLAY OF PLATES',
       'METER EXP.', 'DISPLAY OF TABS', 'PARKED ON SIDEWALK',
       'FIRE HYDRANT', 'NO PARKING', 'WHITE ZONE', 'RESTRICTED TAXI ZONE',
       'DISABLED PARKING/NO DP ID', 'NO STOPPING/ANTI-GRIDLOCK ZONE',
       'PARKED OVER TIME LIMIT', 'NO STOP/STANDING',
       'BLK BIKE PATH OR LANE', 'OUTSIDE LINES/METER',
       'OVNIGHT PRK W/OUT PE', 'PREFERENTIAL PARKING', 'YELLOW ZONE',
       'DOUBLE PARKING', 'PARKED IN BUS ZONE', 'HANDICAP/NO DP ID',
       'STOP/STAND PROHIBIT', 'PARKED IN PARKWAY', '22514',
       'BLOCKING DRIVEWAY', '18 IN. CURB/2 WAY',
       'NO PARKING BETWEEN POSTED HOURS', 'PRIVATE PROPERTY',
       'OFF STR MTR/OUT LINE', 'PARKED IN FIRE LANE',
       'COMM TRAILER/22 FT.', 'DISABLED PARKING/BOUNDARIES',
       'OFF STR/OVERTIME/MTR', 'DP-BLKNG ACCESS RAMP', 'PK OVERSIZ',
       'CARSHARE PARKING', 'SAFETY ZONE/CURB', 'PARKING/FRONT YAR

In [130]:
df[df['Violation Description'] == 'CLEANING VEH/STREET']

,Unnamed: 0,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
17695,1753653,4338918846,2018/09/05 12:00:00 AM,1221.0,NaN,NaN,CA,NaN,OTHR,PA,RD,980 MANZANITA ST,00446,54.0,80.74,CLEANING VEH/STREET,53.0,34.090388,-118.282836
53686,5359303,4291389071,2017/01/13 12:00:00 AM,853.0,NaN,NaN,CA,NaN,HOND,PA,SL,14565 LANARK ST,00300,53.0,80.74,CLEANING VEH/STREET,53.0,34.217941,-118.450313
55191,5509385,4298738312,2016/12/19 12:00:00 AM,1103.0,NaN,NaN,CA,NaN,FORD,CM,BK,2626 ALSACE AVE,00531,55.0,80.74,CLEANING VEH/STREET,53.0,34.032247,-118.355517


In [131]:
df[df['Violation code'] == '80.69BS']['Violation Description']

0        NO PARK/STREET CLEAN
1        NO PARK/STREET CLEAN
2        NO PARK/STREET CLEAN
3        NO PARK/STREET CLEAN
4        NO PARK/STREET CLEAN
                 ...         
87537    NO PARK/STREET CLEAN
87548    NO PARK/STREET CLEAN
87550    NO PARK/STREET CLEAN
87551    NO PARK/STREET CLEAN
87553    NO PARK/STREET CLEAN
Name: Violation Description, Length: 21208, dtype: object

In [132]:
df[df['Violation Description'] == 'NO PARK/STREET CLEAN']

,Unnamed: 0,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
0,28,4361326712,2019/09/17 12:00:00 AM,1205.0,NaN,NaN,CA,NaN,TOYT,PA,GY,309 WINDWARD AVE,00163,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.989412,-118.469024
1,112,4361286823,2019/09/17 12:00:00 AM,853.0,NaN,NaN,CA,NaN,TOYT,PA,WT,934 84TH ST W,00553,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.962435,-118.290126
2,224,4361159515,2019/09/17 12:00:00 AM,833.0,NaN,NaN,CA,NaN,NISS,PA,WT,2601 MONMOUTH AVE,00536,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.032120,-118.284957
3,473,4361237226,2019/09/17 12:00:00 AM,1015.0,NaN,NaN,CA,NaN,HOND,PA,RD,615 HAMPTON DR,00133,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.994284,-118.474775
4,592,4360129196,2019/09/17 12:00:00 AM,1007.0,NaN,NaN,CA,NaN,VOLV,PA,SL,1346 MCCADDEN PL N,00487,54.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.095217,-118.337472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87546,8705209,1111207742,2015/07/02 12:00:00 AM,1018.0,NaN,NaN,CA,NaN,NISS,PA,GY,10717 LAWLER ST,00146,51.0,8069BS,NO PARK/STREET CLEAN,73.0,34.025129,-118.412452
87548,8705581,4262972151,2015/07/02 12:00:00 AM,1010.0,NaN,NaN,WA,NaN,CHEV,PA,RD,4322 LEIMERT BLVD,00541,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.005806,-118.328999
87550,8705782,4262766314,2015/07/02 12:00:00 AM,1302.0,NaN,NaN,CA,NaN,CHEV,PA,GN,5306 VANTAGE AVE,00355,53.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.166789,-118.398139
87551,8705813,4262762140,2015/07/02 12:00:00 AM,1325.0,NaN,NaN,CA,NaN,NISS,PA,BK,8229 CALABAR AVE,00137,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.960751,-118.439362


In [133]:
df[(df['Location'].str.contains('WINDWARD')) & (df['Violation Description'] == 'NO PARK/STREET CLEAN')]

,Unnamed: 0,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
0,28,4361326712,2019/09/17 12:00:00 AM,1205.0,NaN,NaN,CA,NaN,TOYT,PA,GY,309 WINDWARD AVE,00163,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.989412,-118.469024
1370,139182,4358481536,2019/07/22 12:00:00 AM,1140.0,NaN,NaN,AL,NaN,CADI,PA,BK,77 WINDWARD AVE E,112R,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.987515,-118.472533
2307,229728,4357259970,2019/07/01 12:00:00 AM,1110.0,NaN,NaN,NV,NaN,NISS,PA,RD,69 WINDWARD AVE,00136,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.987456,-118.472630
6525,651005,4352300315,2019/04/09 12:00:00 AM,1200.0,NaN,NaN,CA,NaN,LEXS,PA,GY,66 WINDWARD AVE,00135,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.987435,-118.472664
7280,726282,4351612661,2019/03/26 12:00:00 AM,1111.0,NaN,NaN,UT,NaN,HYUN,PA,BL,64 WINDWARD AVE,00132,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.987420,-118.472689
8345,829154,4349764495,2019/03/05 12:00:00 AM,1109.0,NaN,NaN,CA,NaN,MAZD,PA,SL,56 WINDWARD AVE,00132,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.987359,-118.472788
16726,1659384,4340345590,2018/09/25 12:00:00 AM,1138.0,NaN,NaN,CA,NaN,SUBA,PA,BK,56 WINDWARD AVE,161R,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.987359,-118.472788
20207,2010426,4332648950,2018/07/17 12:00:00 AM,1231.0,NaN,NaN,CA,NaN,HOND,PA,WT,325 WINDWARD AVE,00134,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.989715,-118.468564
22779,2270957,4333866143,2018/05/29 12:00:00 AM,1132.0,NaN,NaN,CA,NaN,DODG,PA,SL,36 WINDWARD AVE,00133,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.987207,-118.473037
35531,3536292,4318915086,2017/10/30 12:00:00 AM,1140.0,NaN,NaN,CA,NaN,TOYT,PA,WT,23 WINDWARD AVE,00135,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.987115,-118.473187


In [134]:
pd.Timestamp(df.loc[0, 'Issue Date'])

Timestamp('2019-09-17 00:00:00')

In [135]:
df.loc[:,'Issue Date'] = df['Issue Date'].apply(pd.Timestamp)

In [136]:
df

,Unnamed: 0,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
0,28,4361326712,2019-09-17,1205.0,NaN,NaN,CA,NaN,TOYT,PA,GY,309 WINDWARD AVE,00163,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.989412,-118.469024
1,112,4361286823,2019-09-17,853.0,NaN,NaN,CA,NaN,TOYT,PA,WT,934 84TH ST W,00553,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.962435,-118.290126
2,224,4361159515,2019-09-17,833.0,NaN,NaN,CA,NaN,NISS,PA,WT,2601 MONMOUTH AVE,00536,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.032120,-118.284957
3,473,4361237226,2019-09-17,1015.0,NaN,NaN,CA,NaN,HOND,PA,RD,615 HAMPTON DR,00133,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.994284,-118.474775
4,592,4360129196,2019-09-17,1007.0,NaN,NaN,CA,NaN,VOLV,PA,SL,1346 MCCADDEN PL N,00487,54.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.095217,-118.337472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87549,8705615,4262938105,2015-07-02,1622.0,BH198,NaN,CA,NaN,NISS,PA,SL,200 BREED ST N,00602,56.0,88.13B+,METER EXP.,63.0,34.042926,-118.211886
87550,8705782,4262766314,2015-07-02,1302.0,NaN,NaN,CA,NaN,CHEV,PA,GN,5306 VANTAGE AVE,00355,53.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.166789,-118.398139
87551,8705813,4262762140,2015-07-02,1325.0,NaN,NaN,CA,NaN,NISS,PA,BK,8229 CALABAR AVE,00137,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.960751,-118.439362
87552,8705825,4262762022,2015-07-02,1256.0,NaN,NaN,CA,NaN,NISS,PA,BK,R/O 203 CULVER BLVD E,00137,51.0,80.69AP+,NO STOP/STANDING,93.0,33.958915,-118.448378


In [137]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76138 entries, 0 to 87553
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Unnamed: 0             76138 non-null  int64         
 1   Ticket number          76138 non-null  int64         
 2   Issue Date             76138 non-null  datetime64[ns]
 3   Issue time             76127 non-null  float64       
 4   Meter Id               21218 non-null  object        
 5   Marked Time            2556 non-null   float64       
 6   RP State Plate         76137 non-null  object        
 7   VIN                    0 non-null      float64       
 8   Make                   76107 non-null  object        
 9   Body Style             76098 non-null  object        
 10  Color                  76126 non-null  object        
 11  Location               76138 non-null  object        
 12  Route                  75807 non-null  object        
 13  A

In [138]:
df['Issue Date'].apply(lambda x: x.dayofweek)

0        1
1        1
2        1
3        1
4        1
        ..
87549    3
87550    3
87551    3
87552    3
87553    3
Name: Issue Date, Length: 76138, dtype: int64

In [139]:
df.loc[:,'Day of Week'] = df['Issue Date'].apply(lambda x: x.dayofweek)

In [140]:
df

,Unnamed: 0,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude,Day of Week
0,28,4361326712,2019-09-17,1205.0,NaN,NaN,CA,NaN,TOYT,PA,GY,309 WINDWARD AVE,00163,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.989412,-118.469024,1
1,112,4361286823,2019-09-17,853.0,NaN,NaN,CA,NaN,TOYT,PA,WT,934 84TH ST W,00553,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.962435,-118.290126,1
2,224,4361159515,2019-09-17,833.0,NaN,NaN,CA,NaN,NISS,PA,WT,2601 MONMOUTH AVE,00536,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.032120,-118.284957,1
3,473,4361237226,2019-09-17,1015.0,NaN,NaN,CA,NaN,HOND,PA,RD,615 HAMPTON DR,00133,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.994284,-118.474775,1
4,592,4360129196,2019-09-17,1007.0,NaN,NaN,CA,NaN,VOLV,PA,SL,1346 MCCADDEN PL N,00487,54.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.095217,-118.337472,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87549,8705615,4262938105,2015-07-02,1622.0,BH198,NaN,CA,NaN,NISS,PA,SL,200 BREED ST N,00602,56.0,88.13B+,METER EXP.,63.0,34.042926,-118.211886,3
87550,8705782,4262766314,2015-07-02,1302.0,NaN,NaN,CA,NaN,CHEV,PA,GN,5306 VANTAGE AVE,00355,53.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.166789,-118.398139,3
87551,8705813,4262762140,2015-07-02,1325.0,NaN,NaN,CA,NaN,NISS,PA,BK,8229 CALABAR AVE,00137,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.960751,-118.439362,3
87552,8705825,4262762022,2015-07-02,1256.0,NaN,NaN,CA,NaN,NISS,PA,BK,R/O 203 CULVER BLVD E,00137,51.0,80.69AP+,NO STOP/STANDING,93.0,33.958915,-118.448378,3


In [141]:
week_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
week_names_dict = {index: name for index, name in enumerate(week_names)}

In [142]:
week_names_dict

{0: 'Monday',
 1: 'Tuesday',
 2: 'Wednesday',
 3: 'Thursday',
 4: 'Friday',
 5: 'Saturday',
 6: 'Sunday'}

In [143]:
df.loc[:,'Day of Week'] = df['Day of Week'].replace(week_names_dict)

In [144]:
df

,Unnamed: 0,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude,Day of Week
0,28,4361326712,2019-09-17,1205.0,NaN,NaN,CA,NaN,TOYT,PA,GY,309 WINDWARD AVE,00163,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.989412,-118.469024,Tuesday
1,112,4361286823,2019-09-17,853.0,NaN,NaN,CA,NaN,TOYT,PA,WT,934 84TH ST W,00553,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.962435,-118.290126,Tuesday
2,224,4361159515,2019-09-17,833.0,NaN,NaN,CA,NaN,NISS,PA,WT,2601 MONMOUTH AVE,00536,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.032120,-118.284957,Tuesday
3,473,4361237226,2019-09-17,1015.0,NaN,NaN,CA,NaN,HOND,PA,RD,615 HAMPTON DR,00133,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.994284,-118.474775,Tuesday
4,592,4360129196,2019-09-17,1007.0,NaN,NaN,CA,NaN,VOLV,PA,SL,1346 MCCADDEN PL N,00487,54.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.095217,-118.337472,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87549,8705615,4262938105,2015-07-02,1622.0,BH198,NaN,CA,NaN,NISS,PA,SL,200 BREED ST N,00602,56.0,88.13B+,METER EXP.,63.0,34.042926,-118.211886,Thursday
87550,8705782,4262766314,2015-07-02,1302.0,NaN,NaN,CA,NaN,CHEV,PA,GN,5306 VANTAGE AVE,00355,53.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.166789,-118.398139,Thursday
87551,8705813,4262762140,2015-07-02,1325.0,NaN,NaN,CA,NaN,NISS,PA,BK,8229 CALABAR AVE,00137,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.960751,-118.439362,Thursday
87552,8705825,4262762022,2015-07-02,1256.0,NaN,NaN,CA,NaN,NISS,PA,BK,R/O 203 CULVER BLVD E,00137,51.0,80.69AP+,NO STOP/STANDING,93.0,33.958915,-118.448378,Thursday


In [164]:
def nearby_coords(df, lat, lon, radius=0.0001, st_clean_only=True):
    try:
        if st_clean_only:
            return df[df['Latitude'].between(lat-radius, lat+radius) & df['Longitude']
                      .between(lon-radius, lon+radius) & (df['Violation Description'] == 'NO PARK/STREET CLEAN')]
        else:
            return df[df['Latitude'].between(lat-radius, lat+radius) & df['Longitude'].between(lon-radius, lon+radius)]
    except:
        return

In [169]:
def nearby_coords(df, lat, lon, radius=0.0001, st_clean_only=True):
    if st_clean_only:
        return df[df['Latitude'].between(lat-radius, lat+radius) & df['Longitude']
                  .between(lon-radius, lon+radius) & (df['Violation Description'] == 'NO PARK/STREET CLEAN')]
    else:
        return df[df['Latitude'].between(lat-radius, lat+radius) & df['Longitude'].between(lon-radius, lon+radius)]


In [180]:
nearby_coords(df, 34.039664,-118.322450, 0.002)

,Unnamed: 0,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude,Day of Week
24231,2415443,4331631043,2018-05-03,1052.0,NaN,NaN,AZ,NaN,FORD,TK,WT,3615 21ST ST W,00143,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.037979,-118.323801,Thursday
36198,3601480,4318561332,2017-10-19,832.0,NaN,NaN,NV,NaN,NISS,PA,GY,1823 4TH AVE,00147,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.040634,-118.321266,Thursday
36199,3601481,4318561321,2017-10-19,828.0,NaN,NaN,CA,NaN,NISS,PA,RD,1726 5TH AVE,00147,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.041558,-118.322458,Thursday
44906,4471143,4310082442,2017-05-31,1041.0,NaN,NaN,AL,NaN,NISS,PA,WT,2025 6TH AVENUE,00143,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.038695,-118.323634,Wednesday
78314,7796016,4272088726,2015-12-02,1010.0,NaN,NaN,GA,NaN,CHEV,PA,GY,2000 4TH AVE,00185,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.038934,-118.321259,Wednesday
87553,8706345,4262600580,2015-07-02,1017.0,NaN,NaN,CA,NaN,CHEV,PA,RD,1925 5TH AVE,00171,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.039664,-118.322450,Thursday


In [197]:
nearby_coords(df, 33.989412,-118.469024, .01)[df['Location'].str.contains('WINDWARD') 
                        & (df['Day of Week'] =='Tuesday')][['Location', 'Issue time']].sort_values('Issue time')

/home/greg/anaconda3/envs/parking/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,Location,Issue time
58012,46 WINDWARD AVE,1106.0
48124,32 WINDWARD AVENUE,1107.0
60772,52 WINDWARD AVE,1108.0
8345,56 WINDWARD AVE,1109.0
75290,54 WINDWARD AVENUE,1110.0
7280,64 WINDWARD AVE,1111.0
59393,40 WINDWARD AVE,1114.0
78686,46 WINDWARD AVE E,1117.0
42351,66 WINDWARD AVE,1118.0
22779,36 WINDWARD AVE,1132.0
